### Linking to the google drive to access any data or files

In [2]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the required libraries

In [3]:
# Add the custom module path to sys.path to allow for imports from that directory
import sys
sys.path.append('/content/drive/MyDrive/main/src/')

# Import necessary libraries
import os
import random
import numpy as np
import torch
import torchvision
import torchvision.transforms as T
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

### Setting up the seed and variables

In [4]:
# Set a fixed seed for reproducibility
seed = 26
random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

# Initialise parameters for data
input_size = 256
batch_size = 32
num_workers = 12
norm_arr = ([0.5159, 0.5159, 0.5159], [0.2554, 0.2554, 0.2554])
images_dir = "/content/drive/MyDrive/data/covid_pneumonia"

### Loading the dataset

In [5]:
# Define transformations for data
data_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(norm_arr[0], norm_arr[1])
])

# Load datasets
data_loaders = {
    'train': DataLoader(
        datasets.ImageFolder(os.path.join(images_dir, 'train'), transform=data_transforms),
        batch_size=batch_size, shuffle=True, num_workers=num_workers
    ),
    'val': DataLoader(
        datasets.ImageFolder(os.path.join(images_dir, 'val'), transform=data_transforms),
        batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
}

# Load pre-trained model for feature extraction
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Remove the last fully connected layer to use the model as a feature extractor
feature_extractor = torch.nn.Sequential(*(list(model.children())[:-1]))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 181MB/s]


### Getting the scores and confusion matrix

In [6]:
# Function to extract features
def extract_features(data_loader, feature_extractor, device):
    features = []
    labels = []

    for inputs, targets in data_loader:
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = feature_extractor(inputs)
            outputs = outputs.view(outputs.size(0), -1).cpu().numpy()

        features.extend(outputs)
        labels.extend(targets.numpy())

    return np.array(features), np.array(labels)

# Extract features from the training and validation sets
train_features, train_labels = extract_features(data_loaders['train'], feature_extractor, device)
val_features, val_labels = extract_features(data_loaders['val'], feature_extractor, device)

# Train Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(train_features, train_labels)

# Evaluate Naive Bayes classifier
val_predictions = nb_classifier.predict(val_features)
print("Classification Report:\n", classification_report(val_labels, val_predictions, target_names=data_loaders['train'].dataset.classes))
print("Confusion Matrix:\n", confusion_matrix(val_labels, val_predictions))

Classification Report:
               precision    recall  f1-score   support

       covid       0.84      0.83      0.83       543
      normal       0.81      0.75      0.78       542
       viral       0.78      0.96      0.86       202

    accuracy                           0.82      1287
   macro avg       0.81      0.85      0.82      1287
weighted avg       0.82      0.82      0.82      1287

Confusion Matrix:
 [[449  92   2]
 [ 81 409  52]
 [  4   5 193]]
